In [29]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

import os
import glob

from tqdm import tqdm
from PIL import Image
from IPython import display
%matplotlib inline

Upload the data

In [30]:
path = r'C:\Users\ecath\Desktop\Research\Raw Data\Ex Vivo CSV Files'
os.listdir(path)

['H72_F75_Electrode_KH_OM_KH.csv',
 'H79_F108_csv_annot_OM_KH.csv',
 'H79_F125_csv_annot MISSING H8_OM_KH.csv',
 'H81_F158_csv_annot_KH.csv',
 'H81_F171_csv_annot_OM_KH.csv',
 'H82_F111_csv_annot_OM_KH.csv',
 'H96_F61_csv_annot_OM_KH.csv',
 'H96_F88_csv_annot_OM_KH.csv']

In [31]:
data_list = []
for filename in glob.glob(os.path.join(path, '*.csv')):
    data = pd.read_csv(filename, header=0)
    print(filename, data.shape)
    data_list.append(data)

C:\Users\ecath\Desktop\Research\Raw Data\Ex Vivo CSV Files\H72_F75_Electrode_KH_OM_KH.csv (12207, 384)
C:\Users\ecath\Desktop\Research\Raw Data\Ex Vivo CSV Files\H79_F108_csv_annot_OM_KH.csv (8270, 384)
C:\Users\ecath\Desktop\Research\Raw Data\Ex Vivo CSV Files\H79_F125_csv_annot MISSING H8_OM_KH.csv (8775, 384)
C:\Users\ecath\Desktop\Research\Raw Data\Ex Vivo CSV Files\H81_F158_csv_annot_KH.csv (8138, 384)
C:\Users\ecath\Desktop\Research\Raw Data\Ex Vivo CSV Files\H81_F171_csv_annot_OM_KH.csv (10173, 384)
C:\Users\ecath\Desktop\Research\Raw Data\Ex Vivo CSV Files\H82_F111_csv_annot_OM_KH.csv (10173, 384)
C:\Users\ecath\Desktop\Research\Raw Data\Ex Vivo CSV Files\H96_F61_csv_annot_OM_KH.csv (8191, 384)
C:\Users\ecath\Desktop\Research\Raw Data\Ex Vivo CSV Files\H96_F88_csv_annot_OM_KH.csv (8191, 384)


In [26]:
data_list[0].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240 entries, 0 to 239
Columns: 1919 entries, Unnamed: 0 to Bsk 2 H1_xf.7
dtypes: float64(1918), object(1)
memory usage: 3.5+ MB


In [32]:
drivers = []
nondrivers = []
noises = []

for df in data_list:
    driver = pd.DataFrame()
    nondriver = pd.DataFrame()
    noise = pd.DataFrame()
    for col in df.columns[1::3]:
        curr_index = list(df.columns).index(col)
        prev_index = curr_index - 1
        next_index = curr_index + 1
        prev_col = df.iloc[:,prev_index]
        next_col = df.iloc[:,next_index]
        if df[col][0] == 1:
            driver[df.columns[prev_index]] = prev_col
            driver[df.columns[next_index]] = next_col
        elif df[col][0] == 0:
            nondriver[df.columns[prev_index]] = prev_col
            nondriver[df.columns[next_index]] = next_col
        elif df[col][0] == -1:
            noise[df.columns[prev_index]] = prev_col
            noise[df.columns[next_index]] = next_col
    drivers.append(driver)
    nondrivers.append(nondriver)
    noises.append(noise)

# Fourier transform

In [7]:
# function that delete NaN and fully zero columns
def del_nul_and_nan(df):
    
    df.fillna(value=0, axis=1, inplace=True)
    
    for col in df.columns:
        if df[col].sum() == 0:
            df.drop([col], axis = 1, inplace = True)  
    
    return df

In [8]:
#create DataFrame with signal spectrum cutted on 20Hz for one dataframe from one class

def concat_fft(df, N, T):
    
    fft_out = pd.DataFrame()
    
    for col in df.columns:
        fft_yf = np.fft.fft(df[col]) #spectrum
        fft_xf = np.fft.fftfreq(N, T) #frequencies
        
        fft_20_index = np.argwhere((fft_xf<20) & (fft_xf>0))        
        fft_yf_20 = fft_yf[fft_20_index] #cutting on 20Hz
        fft_xf_20 = fft_xf[fft_20_index] #cutting on 20Hz
        
        fft_yf_20 = pd.DataFrame(np.abs(fft_yf_20)/N, columns=[col + '_yf'])
        fft_xf_20 = pd.DataFrame(fft_xf_20, columns=[col + '_xf']) 

        fft_out = pd.concat([fft_out, fft_yf_20, fft_xf_20], axis=1)
        
    return(fft_out)

In [9]:
def full_fft_list(class_df):
    
    all_fft_el = []
    all_fft_om = []

    for k, df in enumerate(class_df):

        electrode_signal = df[df.columns[::2]]
        optical_signal = df[df.columns[1::2]]

        electrode_signal = del_nul_and_nan(electrode_signal)
        optical_signal = del_nul_and_nan(optical_signal)        

        N = df.shape[0] # Number of sample points
        Fs_el = 1017.25 # sampling rate
        Fs_om = 1000.0
        T_el = 1.0/Fs_el
        T_om = 1.0/Fs_om

        fft_el = concat_fft(electrode_signal, N, T_el)
        all_fft_el.append(fft_el)

        fft_om = concat_fft(optical_signal, N, T_om)
        all_fft_om.append(fft_om)
        
    return(all_fft_el, all_fft_om)

In [10]:
def fft_creation_el(dataframe, label):
    
    all_fft_el, _ = full_fft_list(dataframe)

    for i in range(len(all_fft_el)):
        
        target = np.full(shape=(1, all_fft_el[i].shape[1]), fill_value = label)
        target = pd.DataFrame(target, columns=all_fft_el[i].columns, index=['target'])

        all_fft_el[i] = all_fft_el[i].append(target)
    
    all_fft = pd.DataFrame()
    
    for i in range(len(all_fft_el)):
        all_fft = pd.concat([all_fft, all_fft_el[i]], axis=1)

    return(all_fft)

In [11]:
def fft_creation_om(dataframe, label):
   
    _, all_fft_om = full_fft_list(dataframe)
    
#     m = []
    
#     for i in range(len(all_fft_om)):
#         m.append(all_fft_om[i].shape[0])

    for i in tqdm(range(len(all_fft_om))):
#         mx = np.max(m) - all_fft_om[i].shape[0]

#         zeros = np.zeros(shape=(mx, all_fft_om[i].shape[1]))
#         zeros = pd.DataFrame(zeros, columns=all_fft_om[i].columns, index=np.arange(all_fft_om[i].shape[0], np.max(m)))
        
#         all_fft_om[i] = all_fft_om[i].append(zeros)

        target = np.full(shape=(1, all_fft_om[i].shape[1]), fill_value = label)
        target = pd.DataFrame(target, columns=all_fft_om[i].columns, index=['target'])

        all_fft_om[i] = all_fft_om[i].append(target)
    
    all_fft = pd.DataFrame()
    
    for i in range(len(all_fft_om)):
        all_fft = pd.concat([all_fft, all_fft_om[i]], axis=1)

    return(all_fft)

In [12]:
path = r'C:\Users\Public\Documents\HRS'

In [13]:
a = fft_creation_el(drivers, 1)
b = fft_creation_el(nondrivers, 0)
c = fft_creation_el(noises, -1) 

final = pd.concat([a, b, c], axis=1)
final.fillna(value=0, axis=1, inplace=True)
final.to_csv(path + '\All FFT of electrode signal.csv', sep=',', index=True)

C:\Users\ecath\Anaconda3\lib\site-packages\pandas\core\frame.py:3035: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)
C:\Users\ecath\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [14]:
a = fft_creation_om(drivers, 1)
b = fft_creation_om(nondrivers, 0)
c = fft_creation_om(noises, -1) 

final = pd.concat([a, b, c], axis=1)
final.fillna(value=0, axis=1, inplace=True)
final.to_csv(path + '\All FFT of optical signal.csv', sep=',', index=True)

C:\Users\ecath\Anaconda3\lib\site-packages\pandas\core\frame.py:3035: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)
C:\Users\ecath\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
